# Covid-19 Genome analysis using Biopython

### Fetch COVID-19 Genome from NCBI
**MN908947** - covid-19 genome used here was sequenced from a sample of bronchoalveolar lavage fluid from a single patient who was a worker at the market and who was admitted to the Central Hospital of Wuhan on 26 December 2019